Initializing the python libraries to the notebook

In [ ]:
import pandas as pd
import numpy as np

To initialize the train data set into our python program we need to use pandas to first read the csv and to perform operation on train.csv we use pandas-dataframe and creates an object of it.

In [ ]:
df = pd.DataFrame(pd.read_csv('/content/train_data_set/train.csv'))
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


df.shape is used to get number of rows and columns present in the csv file or in a dataframe.

In [ ]:
df.shape

(891, 12)

To check whether there are null values in our csv file, we use functions of dataframe such as null() and to count total number of null values in each row we use sum(), as shown in below code.

In [ ]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Seperating out the columns which have more than 35% of the null values missing in the dataset.

In [ ]:
# df.isnull().sum() returms a pandas series with column name as the label index
# and total count of null values in the coulmn as it's value
# and we are storing only those columns which have more than 35% of the data missing

r = df.isnull().sum()

drop_col = r[r > (35/100 * df.shape[0])]
drop_col

Cabin    687
dtype: int64

so in above result "cabin" column was the one having empty / null values more than 35%.
Keeping cabin column is no use to us because null are more than data, so neglecting the column "cabin"

In [ ]:
drop_col.index

Index(['Cabin'], dtype='object')

In [ ]:
df.drop(drop_col.index, axis=1, inplace=True)
df.shape
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64

In [ ]:
 df.fillna(df.mean(), inplace=True)
 df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
dtype: int64

Now a column "Embarked" is remained with two null values, beacuse of which Embarked contains string values fillna() was not able to calculate mean of string values.

In [ ]:
df['Embarked'].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

For the null rows of Embarked coluumn, we fill the NULL values with the most frequent value used in the column i.e.. "S".

In [ ]:
df["Embarked"].fillna("S", inplace=True)

In [ ]:
df.isnull().sum()     
# Now all the NULL values in our dataframe are been removed

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [ ]:
df.corr()
# this above line of code returns the correlation between 
# every column with the every column

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.033207,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.069809,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.331339,0.083081,0.018443,-0.549500
Age,0.033207,-0.069809,-0.331339,1.000000,-0.232625,-0.179191,0.091566
SibSp,-0.057527,-0.035322,0.083081,-0.232625,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.179191,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.091566,0.159651,0.216225,1.000000


From the above result table, we can easily conclude 
1. the relation between Fare and Pclass i.e.. as per the Pclass the fare was different, such as Pclass1 with high fare, Pclass2 with less fare than Pclass1 and Pclass3 with lesser fare than both the other Pclass fares.

2. We also see the relation between the age and the Pclass which gives conclusion such as passenger which high age were in lower class Pclass3.
---



sibsp : Number of siblings / Spouse Abroad

parch : Number of parents / Childern Abroad

So we can make a new column down with the name family_size by combining these two columns.

In [ ]:
df['FamilySize'] = df['SibSp'] + df['Parch']
df.drop(['SibSp', 'Parch'], axis=1, inplace = True)
df.corr()

,PassengerId,Survived,Pclass,Age,Fare,FamilySize
PassengerId,1.000000,-0.005007,-0.035144,0.033207,0.012658,-0.040143
Survived,-0.005007,1.000000,-0.338481,-0.069809,0.257307,0.016639
Pclass,-0.035144,-0.338481,1.000000,-0.331339,-0.549500,0.065997
Age,0.033207,-0.069809,-0.331339,1.000000,0.091566,-0.248512
Fare,0.012658,0.257307,-0.549500,0.091566,1.000000,0.217138
FamilySize,-0.040143,0.016639,0.065997,-0.248512,0.217138,1.000000


**From the above correlation we can conclude that :**
1. Between Pclass and Survived relation, we see that the less Pclass have high survive rate and high Pclass have less survive rate and the realtion with Survived with the Fare says that the passengers with high fare price has high survival ratio

concluding with above two different relation we can say that the Pclass with moderate fare price have high survival rate than other passengers.

2. Relation betn FamilySize and Survived, does not have much correlance with survival rate.


Lets check wether the person was alone or not and wether it affect the survival rate or not

In [ ]:
df['Alone'] = [0 if df['FamilySize'][i] > 0 else 1 for i in df.index]
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Embarked,FamilySize,Alone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,S,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,7.9250,S,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,S,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,S,0,1


In [ ]:
df.groupby(['Alone'])['Survived'].mean()

Alone
0    0.505650
1    0.303538
Name: Survived, dtype: float64

Frome above result we conclude that,
if the person was alone then there has less chance of survival rate

---
The reason might be the person who is travelling with his family might be belonging to rich class and might be prioritized over other.


In [ ]:
df[['Alone', 'Fare']].corr()

,Alone,Fare
Alone,1.000000,-0.271832
Fare,-0.271832,1.000000


So we can see if the person is not alone, then he/she must be having high fare price and the survival rate of that person would also be high.

In [ ]:
df.groupby(['Sex'])['Survived'].mean()

Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64

It shows that the female passengers have more chances of the survival that the male passengers.

It shows women are prioritized over men.

In [ ]:
df.groupby(['Embarked'])['Survived'].mean()

Embarked
C    0.553571
Q    0.389610
S    0.339009
Name: Survived, dtype: float64

We see that people having the embarkation port as "C" has high survival rate than others.

# New Section

**CONCLUSION**

---
*  Female passengers were prioritized over men
*  Passengers travelling with their family have higher survival rate.
*  Passengers who are borded the ship at Cherbourg, survived more in proportion then the others.
*  People with higher class or rich people have higher survival rate than others. 



